# Notebook for comparing the empirically derived f factor and that derived from the E/I method for Cut Face Sandstone

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)

import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag

import scipy.stats as st
import matplotlib.pyplot as plt
from IPython.display import display
import matplotlib as mpl
from matplotlib import cm
import math as math

%config InlineBackend.figure_format = 'retina'

## Load Cut Face demagnetization data

In [3]:
Cutface_specimens = pd.read_csv('../../data/Pmag/cutface/specimens.txt', sep='\t', header=1)
specimens_mt_tc = Cutface_specimens[(Cutface_specimens['dir_comp']=='mt') & (Cutface_specimens['dir_tilt_correction']==100)]
specimens_ht_tc = Cutface_specimens[(Cutface_specimens['dir_comp']=='ht') & (Cutface_specimens['dir_tilt_correction']==100)]

In [4]:
mean_mt = ipmag.fisher_mean(specimens_mt_tc['dir_dec'].tolist(),specimens_mt_tc['dir_inc'].tolist())
mean_mt

{'dec': 286.49699313337345,
 'inc': 42.003560540334476,
 'n': 167,
 'r': 163.55867702232504,
 'k': 48.23726255190197,
 'alpha95': 1.5861242493209464,
 'csd': 11.662554654251615}

In [5]:
mean_ht = ipmag.fisher_mean(specimens_ht_tc['dir_dec'].tolist(),specimens_ht_tc['dir_inc'].tolist())
mean_ht

{'dec': 286.57662835036626,
 'inc': 29.430252935780295,
 'n': 157,
 'r': 152.63958083558612,
 'k': 35.77637702199419,
 'alpha95': 1.9070660543868,
 'csd': 13.542125665843008}

## Load North Shore Volcanic Group site level paleodirection data

In [6]:
NSVG_Data=pd.read_csv('../../data/data_Compiled/Tauxe2009/pmag_results.txt',sep='\t',skiprows=1)
nneu_site_list = ['ns002',
                  'ns003',
                  'ns004',
                  'ns005',
                  'ns016',
                  'ns018',
                  'ns019',
                  'ns020',
                  'ns021',
                  'ns022',
                  'ns023',
                  'ns028',
                  'ns030',
                  'ns031',
                  'ns032']
nneu_data = NSVG_Data.loc[NSVG_Data['er_site_names'].isin(nneu_site_list)]
Books1972_sites = pd.read_csv('../../data/data_compiled/Books1972/sites.txt',sep='\t',header=1)
Books1972_MN_sites = Books1972_sites[Books1972_sites.location == 'North Shore Volcanic Group:Minnesota']
nneu_nmil_sites_B72 = ['NS269','NS378','NS227']
nneu_nkcr_sites_B72 = ['NS229','NS375']
nneu_nrcb_sites_B72 = ['NS226']
nneu_ncvb_sites_B72 = ['NS362','NS365']
nneu_ngha_sites_B72 = ['NS367','NS265']
nneu_ntpb_sites_B72 = ['NS368',
                       'NS369',
                       'NS374',
                       'NS376',
                       'NS377',
                       'NS169',
                       'NS170',
                       'NS171']
nneu_norl_sites_B72 = ['NS370',
                       'NS371',
                       'NS372']
nneu_B72_site_list = nneu_nmil_sites_B72 + nneu_nkcr_sites_B72 + nneu_nrcb_sites_B72 + nneu_ncvb_sites_B72 + nneu_ngha_sites_B72 + nneu_norl_sites_B72 
nneu_data_B72 = Books1972_MN_sites.loc[Books1972_MN_sites['site'].isin(nneu_B72_site_list)]
nneu_combined_dir_dec = nneu_data_B72.dir_dec.tolist() + nneu_data.average_dec.tolist()
nneu_combined_dir_inc = nneu_data_B72.dir_inc.tolist() + nneu_data.average_inc.tolist()


nneu_dirs = ipmag.make_di_block(nneu_combined_dir_dec,
                                nneu_combined_dir_inc)
nneu_dir_mean=pmag.fisher_mean(nneu_dirs)
nneu_dir_mean

{'dec': 291.3242830230793,
 'inc': 40.25007073978004,
 'n': 28,
 'r': 27.15979233643242,
 'k': 32.134912796862814,
 'alpha95': 4.883450101163713,
 'csd': 14.288822983259386}

## common mean test for CF-MT vs CF-HT (with f factors ranging from 0.1 to 1)

In [7]:
mt_directions_block=ipmag.make_di_block(specimens_mt_tc['dir_dec'].tolist(),specimens_mt_tc['dir_inc'].tolist())
ht_directions_block=ipmag.make_di_block(specimens_ht_tc['dir_dec'].tolist(),specimens_ht_tc['dir_inc'].tolist())

spec_ht_inc=specimens_ht_tc['dir_inc'].tolist()
spec_ht_dec=specimens_ht_tc['dir_dec'].tolist()

In [7]:
f_factors = np.arange(0.1,1,.005)

all_f_factors_mt = []
all_angles_mt = []

for f_fact in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, mt_directions_block,NumSims=100,print_result=False)
    all_f_factors_mt.append(f_fact)
    all_angles_mt.append(watson_common_mean[1])



In [8]:
#all_f_factors_mt=np.array(all_f_factors_mt)
#np.savetxt('../code_output/all_factors_mt.txt',all_f_factors_mt)
#all_angles_f_mt=np.array(all_angles_mt)
#np.savetxt('../code_output/all_angles_mt.txt',all_angles_f_mt)

## common mean test for CF-MT vs CF-HT (with f factors ranging from 0.5 to 0.7)

common mean test between the mid-temperature component and the high-temperature component of Cut Face directions

This method is for empirically deriving the f factor by interating through a range of values for f to unsquish the high-temperature component of specimen directions and use the Watson commen mean test for judging whether the resulting directions share a common mean with the mid-temperature directions which are interepreted to be CRMs and do not suffer from inclination shallowing.

In [9]:
f_factors = np.arange(0.5,0.7,.001)

successful_f_factors_mt = []
successful_angles_mt = []

for f_fact in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, mt_directions_block,NumSims=300,print_result=False)
    if watson_common_mean[0] == 1:
        successful_f_factors_mt.append(f_fact)
        successful_angles_mt.append(watson_common_mean[1])

In [10]:
#good_f_factors_mt=np.array(successful_f_factors_mt)
#np.savetxt('../code_output/good_f_factors_mt.txt',good_f_factors_mt)
#good_angles_mt=np.array(successful_angles_mt)
#np.savetxt('../code_output/good_angles_mt.txt',good_angles_mt)

## common mean test for CF-MT vs CF-HT - high resolution (with f factors ranging from 0.5 to 0.7)

In [9]:
f_factors = np.arange(0.1,1,.001)

all_001__f_factors_mt = []
all_001_angles_mt = []

for f_fact in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, mt_directions_block,NumSims=300,print_result=False)
    all_001__f_factors_mt.append(f_fact)
    all_001_angles_mt.append(watson_common_mean[1])

In [10]:
all_001__f_factors_mt=np.array(all_001__f_factors_mt)
np.savetxt('../code_output/all_001_f_factors_mt.txt',all_001__f_factors_mt)
all_001_angles_mt=np.array(all_001_angles_mt)
np.savetxt('../code_output/all_001_angles_mt.txt',all_001_angles_mt)

## common mean test for NSVG vs CF-HT (with f factors ranging from 0.1 to 1)


In [11]:
f_factors = np.arange(0.1,1,.005)

all_factors_nsvg = []
all_angles_nsvg = []

for f_fact_nsvg in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact_nsvg)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, nneu_dirs,NumSims=100,print_result=False)
    all_factors_nsvg.append(f_fact_nsvg)
    all_angles_nsvg.append(watson_common_mean[1])

In [12]:
#all_factors_nsvg=np.array(all_factors_nsvg)
#np.savetxt('../code_output/all_factors_nsvg.txt',all_factors_nsvg)
#all_angles_nsvg=np.array(all_angles_nsvg)
#np.savetxt('../code_output/all_angles_nsvg.txt',all_angles_nsvg)

## common mean test for NSVG vs CF-HT (with f factors ranging from 0.5 to 0.8)

common mean test between the North Shore Volcanic Group directions that do not suffer from inclination shallowing and the high-temperature component of Cut Face directions.

In [13]:
f_factors = np.arange(0.5,0.8,.001)

successful_f_factors_nsvg = []
successful_angles_nsvg = []

for f_fact_nsvg in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact_nsvg)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, nneu_dirs,NumSims=300,print_result=False)
    if watson_common_mean[0] == 1:
        successful_f_factors_nsvg.append(f_fact_nsvg)
        successful_angles_nsvg.append(watson_common_mean[1])

In [14]:
#successful_f_factors_nsvg=np.array(successful_f_factors_nsvg)
#np.savetxt('../code_output/good_f_factors_nsvg.txt',successful_f_factors_nsvg)
#good_angles_mt=np.array(successful_angles_nsvg)
#np.savetxt('../code_output/good_angles_nsvg.txt',successful_angles_nsvg)

### Common mean test for NSVG vs CF-HT - high resolution 0.1 to 1

In [ ]:
f_factors = np.arange(0.1,1,.001)

all_001_f_factors_nsvg = []
all_001_angles_nsvg = []

for f_fact_nsvg in f_factors:

    ht_unsquished_incs = ipmag.unsquish(spec_ht_inc, f_fact_nsvg)
    ht_unsquished_directions_block = ipmag.make_di_block(spec_ht_dec,ht_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(ht_unsquished_directions_block, nneu_dirs,NumSims=300,print_result=False)
    all_001_f_factors_nsvg.append(f_fact_nsvg)
    all_001_angles_nsvg.append(watson_common_mean[1])

In [ ]:
all_001_f_factors_nsvg=np.array(all_001_f_factors_nsvg)
np.savetxt('../code_output/all_001_f_factors_nsvg.txt',all_001_f_factors_nsvg)
all_001_angles_nsvg=np.array(all_001_angles_nsvg)
np.savetxt('../code_output/all_001_angles_nsvg.txt',all_001_angles_nsvg)

## Performing empirical commen mean method and E/I method in context of grain sizes of Cut Face sandstone

In [7]:
cf_grain_sizes=pd.read_csv('../../data/Pmag/cf_specimen_lithologies.csv')
cf_specimens_directions_and_grain=pd.merge(cf_grain_sizes,Cutface_specimens,on='specimen')

In [8]:
cf_fm_grain=cf_specimens_directions_and_grain[(cf_specimens_directions_and_grain['primary_grain_size']>=3) & 
                                                  (cf_specimens_directions_and_grain['dir_comp']=='ht')&
                                                  (cf_specimens_directions_and_grain['dir_tilt_correction']==100)]

cf_vfs_grain=cf_specimens_directions_and_grain[(cf_specimens_directions_and_grain['primary_grain_size']==2) & 
                                               (cf_specimens_directions_and_grain['dir_comp']=='ht')&
                                               (cf_specimens_directions_and_grain['dir_tilt_correction']==100)]

cf_silt_grain=cf_specimens_directions_and_grain[(cf_specimens_directions_and_grain['primary_grain_size']<=1) & 
                                                (cf_specimens_directions_and_grain['dir_comp']=='ht')&
                                                (cf_specimens_directions_and_grain['dir_tilt_correction']==100)]

### Clay and Silt empirical f
Watson common mean test for NSVG vs CF fine-grained specimen directions

In [8]:
cf_silt_grain_incs=cf_silt_grain['dir_inc']
cf_silt_grain_decs=cf_silt_grain['dir_dec']

In [10]:
f_factors = np.arange(0.3,0.7,.005)

good_factors_silt_grain = []
good_angles_silt_grain = []

for f_fact_nsvg in f_factors:

    silt_grain_unsquished_incs = ipmag.unsquish(cf_silt_grain_incs.tolist(), f_fact_nsvg)
    silt_grain_unsquished_directions_block = ipmag.make_di_block(cf_silt_grain_decs.tolist(),silt_grain_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(silt_grain_unsquished_directions_block, nneu_dirs,NumSims=100,print_result=False)
    if watson_common_mean[0] == 1: 
        good_factors_silt_grain.append(f_fact_nsvg)
        good_angles_silt_grain.append(watson_common_mean[1])

In [11]:
print(round(good_factors_silt_grain[good_angles_silt_grain.index(min(good_angles_silt_grain))],4),'for specimens predominantly clay and silt')

0.56 for specimens predominantly clay and silt


In [12]:
np.savetxt('../code_output/good_f_factors_silt.txt',np.array(good_factors_silt_grain))
np.savetxt('../code_output/good_angles_silt.txt',np.array(good_angles_silt_grain))

### Very fine sand grain f

Watson common mean test for NSVG vs CF very fine-grained specimen directions

In [9]:
cf_vf_grain_incs=cf_vfs_grain['dir_inc']
cf_vf_grain_decs=cf_vfs_grain['dir_dec']

In [10]:
f_factors = np.arange(0.4,.8,.005)

good_factors_vf_grain = []
good_angles_vf_grain = []

for f_fact_nsvg in f_factors:

    cf_vf_unsquished_incs = ipmag.unsquish(cf_vf_grain_incs.tolist(), f_fact_nsvg)
    cf_vf_unsquished_directions_block = ipmag.make_di_block(cf_vf_grain_decs.tolist(),cf_vf_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(cf_vf_unsquished_directions_block, nneu_dirs,NumSims=100,print_result=False)
    if watson_common_mean[0] == 1: 
        good_factors_vf_grain.append(f_fact_nsvg)
        good_angles_vf_grain.append(watson_common_mean[1])

In [11]:
print(round(good_factors_vf_grain[good_angles_vf_grain.index(min(good_angles_vf_grain))],4),'for specimens predominantly very fine sand')

0.66 for specimens predominantly very fine sand


In [12]:
np.savetxt('../code_output/good_f_factors_vf.txt',np.array(good_factors_vf_grain))
np.savetxt('../code_output/good_angles_vf.txt',np.array(good_angles_vf_grain))

### fine and medium sand f

In [13]:
cf_fm_grain_incs=cf_fm_grain['dir_inc']
cf_fm_grain_decs=cf_fm_grain['dir_dec']

In [14]:
f_factors = np.arange(0.55,.9,.005)

good_factors_fm_grain = []
good_angles_fm_grain = []

for f_fact_nsvg in f_factors:

    cf_fm_unsquished_incs = ipmag.unsquish(cf_fm_grain_incs.tolist(), f_fact_nsvg)
    cf_fm_unsquished_directions_block = ipmag.make_di_block(cf_fm_grain_decs.tolist(),cf_fm_unsquished_incs)
    watson_common_mean = ipmag.common_mean_watson(cf_fm_unsquished_directions_block, nneu_dirs,NumSims=500,print_result=False)
    if watson_common_mean[0] == 1: 
        good_factors_fm_grain.append(f_fact_nsvg)
        good_angles_fm_grain.append(watson_common_mean[1])

In [15]:
print(round(good_factors_fm_grain[good_angles_fm_grain.index(min(good_angles_fm_grain))],4),'for specimens predominantly fine and medium sand')

0.74 for specimens predominantly fine and medium sand


In [16]:
np.savetxt('../code_output/good_f_factors_fm.txt',np.array(good_factors_fm_grain))
np.savetxt('../code_output/good_angles_fm.txt',np.array(good_angles_fm_grain))

In [21]:
np.tan(np.radians(41))/np.tan(np.radians(58))

0.5431906404825989

In [22]:
np.tan(np.radians(39))/np.tan(np.radians(58))

0.5060092240090853